In [13]:
import daw_readout
import process_data
import pandas as pd
import numpy as np
import h5py
from tqdm import tqdm
import sys
import time
import os

In [14]:
#file_list = 'runlist/LED_20240826_1.7V_17ns'
file_list = 'runlist/runlist'

In [17]:
path_save =""
gain_lv2414 = 77.791   # convertered to ADC  
gain_lv2415 = 96.99    # convertered to ADC  
attenuation_factor_9DB = 2.76   # attenuation factor 
attenuation_factor_6DB = 1.87   # attenuation factor 
attenuation_factor_12DB = 3.85  # attenuation factor 
attenuation_factor_18DB = 7.62   # attenuation factor 
with open(file_list, 'r') as list:
    for line in list:    
        rawfilename = line.rstrip('\n')#[17 :] 
        #file_tag = line.rstrip('\n').rstrip('.bin')[24:][: -12] 
        #print(file_tag)
        #break   
        rawdata = daw_readout.DAWDemoWaveParser(rawfilename)     
        winfo =[]
        for wave in tqdm(rawdata) :            
            ch = wave.Channel
            ttt = wave.Timestamp
            base = wave.Baseline
            pulse = wave.Waveform
            st, minp, ed = process_data.pusle_index(pulse)
            start, lowp, end = process_data.find_waveform_intersections(pulse, base, true, 0.1)
            ht = base - wave.Waveform[minp]          
            area = process_data.pulse_area(pulse, st, ed, base)
            area_fix_range = process_data.pulse_area_fix_range(pulse, 90, 300, base)
            area_fix_range_dy = process_data.pulse_area_fix_range(pulse, 90, 300, base)
            if ch == 0:
                ht = ht * attenuation_factor_9DB
                area_pe = area / (gain_lv2414 *attenuation_factor_9DB)
                area_fix_range_pe = area_fix_range / (gain_lv2414 *attenuation_factor_9DB)
            if ch == 1:
                area_pe = area / gain_lv2415
                area_fix_range_pe = area_fix_range / gain_lv2415
            if ch == 2:
                area_pe = area / gain_lv2414
                area_fix_range_pe = area_fix_range_dy / gain_lv2414 
            winfo.append({         
                #'EvtID': i,
                'Ch':ch,
                'TTT':ttt,   ## Trigger time tag
                'Baseline': base, 
                'Hight': ht, 
                'St': st,
                'Ed': ed,
                'Minp': minp,
                'Width': ed - st,  
                'Area': area_pe,
                'Area_fixrange':area_fix_range_pe,
                'Wave': pulse
            })            
        #file_tag = line.rstrip('\n')[17 :].rstrip('.bin')[24:][: -12]  
        file_tag = line.rstrip('\n').rstrip('.bin')[24:][: -12]  
        path_save = "outnpy/{}.h5df".format(file_tag)
        df = pd.DataFrame(winfo)
        process_data.write_to_hdf5(df, path_save)

376848it [00:37, 10137.35it/s]


h5 Write Time: 1.85 s 
h5 File Size: 235.73 MB
Save to outnpy/lv2415_anode_lv2414_dual_20241012_LED_3.540v_900mv_600ns_attenuator_9DB_400_ratio_run0.h5df


In [16]:
#df = pd.DataFrame(winfo)
#data_array = df.values
#np.save(path_save, data_array)